In [1]:
import numpy as np
import math 
from scipy.stats import norm
from scipy.stats import truncnorm
import scipy.stats as stats

In [57]:
def autoselect_scales_mix_norm(betahat, sebetahat, max_class=None, mult=2):
    sigmaamin = np.min(sebetahat) / 10
    if np.all(betahat**2 < sigmaamin**2):  # Fix the typo and ensure logical comparison
        sigmaamax = 8 * sigmaamin
    else:
        sigmaamax = 2*np.sqrt(np.max(betahat**2 - sebetahat**2))
    
    if mult == 0:
        out = np.array([0, sigmaamax / 2])
    else:
        npoint = math.ceil(math.log2(sigmaamax / sigmaamin) / math.log2(mult))

        # Generate the sequence (-npoint):0 using np.arange
        sequence = np.arange(-npoint, 1)

        # Calculate the output
        out = np.concatenate(([0], (1/mult) ** (-sequence) * sigmaamax))
        if max_class!=None:
            # Check if the length of out is equal to max_class
            if len(out) != max_class:
            # Generate a sequence from min(out) to max(out) with length max_class
                out = np.linspace(np.min(out), np.max(out), num=max_class)
        
    
    return out
     
def autoselect_scales_mix_exp(betahat, sebetahat, max_class=None , mult=1.5,tt=1.5):
    sigmaamin = np.min(sebetahat) / 10
    if np.all(betahat**2 < sigmaamin**2):  # Fix the typo and ensure logical comparison
        sigmaamax = 8 * sigmaamin
    else:
        sigmaamax = tt*np.sqrt(np.max(betahat**2  ))
    
    if mult == 0:
        out = np.array([0, sigmaamax / 2])
    else:
        npoint = math.ceil(math.log2(sigmaamax / sigmaamin) / math.log2(mult))

        # Generate the sequence (-npoint):0 using np.arange
        sequence = np.arange(-npoint, 1)

        # Calculate the output
        out = np.concatenate(([0], (1/mult) ** (-sequence) * sigmaamax))
        if max_class!=None:
            # Check if the length of out is equal to max_class
            if len(out) != max_class:
            # Generate a sequence from min(out) to max(out) with length max_class
                out = np.linspace(np.min(out), np.max(out), num=max_class)
                if(out[2] <1e-2 ):
                 out[2: ] <- out[2: ] +1e-2
         
    
    return out

    
def wpost_exp ( x, s, w, scale):
    print(w)
    if  w[0]==1:
     out =  np.concatenate(([1]  ,np.full( scale.shape[0],[0])))
     return out
    else:
     a=1/scale[1:]
     w = assignment
     a = 1 / scale[1:]  # Note: slicing in Python is zero-based, so [1:] starts from the second element
     lf = norm.logpdf(x, loc=0, scale=s)
     lg = np.log(a) + s**2 * a**2 / 2 - a * x + norm.logcdf(x / s - s * a)
     log_prob = np.concatenate(([lf]  ,lg ))
     bmax=np.max(log_prob)
     log_prob = log_prob - bmax
 
     log_prob = log_prob - bmax
     wpost = w* np.exp( log_prob) / (sum(w *np.exp(log_prob)))
     return wpost    
 
 

def do_truncnorm_argchecks(a, b):
    # Ensure a and b are numpy arrays, even if they are scalars
    a = np.atleast_1d(np.asarray(a))
    b = np.atleast_1d(np.asarray(b))
    if len(a) != len(b):
        raise ValueError("truncnorm functions require that a and b have the same length.")
    if np.any(b < a):
        raise ValueError("truncnorm functions require that a <= b.")
    return a, b

def logscale_sub(logx, logy, epsilon=1e-10):
    diff = logx - logy
    if np.any(diff < 0):
        bad_idx = diff < 0
        logx[bad_idx] = logy[bad_idx]
        print(f"logscale_sub encountered negative value(s) of logx - logy (min: {np.min(diff[bad_idx]):.2e})")
    
    scale_by = logx.copy()
    scale_by[np.isinf(scale_by)] = 0
    return np.log(np.exp(logx - scale_by) - np.exp(logy - scale_by) + epsilon) + scale_by

def my_etruncnorm(a, b, mean=0, sd=1):
    a, b = do_truncnorm_argchecks(a, b)
    
    alpha = (a - mean) / sd
    beta = (b - mean) / sd
    
    flip = (alpha > 0) & (beta > 0) | (beta > np.abs(alpha))
    orig_alpha = alpha.copy()
    alpha[flip] = -beta[flip]
    beta[flip] = -orig_alpha[flip]
    
    dnorm_diff = logscale_sub(stats.norm.logpdf(beta), stats.norm.logpdf(alpha))
    pnorm_diff = logscale_sub(stats.norm.logcdf(beta), stats.norm.logcdf(alpha))
    scaled_res = -np.exp(dnorm_diff - pnorm_diff)
    
    endpts_equal = np.isinf(pnorm_diff)
    scaled_res[endpts_equal] = (alpha[endpts_equal] + beta[endpts_equal]) / 2
    
    lower_bd = np.maximum(beta + 1 / beta, (alpha + beta) / 2)
    bad_idx = (~np.isnan(beta)) & (beta < 0) & ((scaled_res < lower_bd) | (scaled_res > beta))
    scaled_res[bad_idx] = lower_bd[bad_idx]
    
    scaled_res[flip] = -scaled_res[flip]
    
    res = mean + sd * scaled_res
    
    if np.any(sd == 0):
        a = np.tile(a, len(res))
        b = np.tile(b, len(res))
        mean = np.tile(mean, len(res))
        
        sd_zero = (sd == 0)
        res[sd_zero & (b <= mean)] = b[sd_zero & (b <= mean)]
        res[sd_zero & (a >= mean)] = a[sd_zero & (a >= mean)]
        res[sd_zero & (a < mean) & (b > mean)] = mean[sd_zero & (a < mean) & (b > mean)]
    
    return res

def my_e2truncnorm(a, b, mean=0, sd=1):
    a, b = do_truncnorm_argchecks(a, b)
    
    mean = np.atleast_1d(mean)
    sd   = np.atleast_1d(sd)
    alpha = (a - mean) / sd
    beta = (b - mean) / sd
    
    flip = (alpha > 0) & (beta > 0)
    orig_alpha = alpha.copy()
    alpha[flip] = -beta[flip]
    beta[flip] = -orig_alpha[flip]
    
    if np.any(mean != 0):
         
        mean  =  abs(mean)
    
    pnorm_diff = logscale_sub(stats.norm.logcdf(beta), stats.norm.logcdf(alpha))
    alpha_frac = alpha * np.exp(stats.norm.logpdf(alpha) - pnorm_diff)
    beta_frac = beta * np.exp(stats.norm.logpdf(beta) - pnorm_diff)
    
    # Handle inf and nan values in alpha_frac and beta_frac
    alpha_frac[np.isnan(alpha_frac) | np.isinf(alpha_frac)] = 0
    beta_frac[np.isnan(beta_frac) | np.isinf(beta_frac)] = 0
    
    scaled_res = np.ones_like(alpha)
    scaled_res[np.isnan(flip)] = np.nan
    
    alpha_idx = np.isfinite(alpha)
    scaled_res[alpha_idx] = 1 + alpha_frac[alpha_idx]
    beta_idx = np.isfinite(beta)
    scaled_res[beta_idx] -= beta_frac[beta_idx]
    
    endpts_equal = np.isinf(pnorm_diff)
    scaled_res[endpts_equal] = ((alpha[endpts_equal] + beta[endpts_equal]) ** 2) / 4
    
    upper_bd1 = beta ** 2 + 2 * (1 + 1 / beta ** 2)
    upper_bd2 = (alpha ** 2 + alpha * beta + beta ** 2) / 3
    upper_bd = np.minimum(upper_bd1, upper_bd2)
    bad_idx = (~np.isnan(beta)) & (beta < 0) & ((scaled_res < beta ** 2) | (scaled_res > upper_bd))
    scaled_res[bad_idx] = upper_bd[bad_idx]
    
    res = mean ** 2 + 2 * mean * sd * my_etruncnorm(alpha, beta) + sd ** 2 * scaled_res
    
    if np.any(sd == 0):
        a = np.tile(a, len(res))
        b = np.tile(b, len(res))
        mean = np.tile(mean, len(res))
        
        sd_zero = (sd == 0)
        res[sd_zero & (b <= mean)] = b[sd_zero & (b <= mean)] ** 2
        res[sd_zero & (a >= mean)] = a[sd_zero & (a >= mean)] ** 2
        res[sd_zero & (a < mean) & (b > mean)] = mean[sd_zero & (a < mean) & (b > mean)] ** 2
    
    return res


class PosteriorMeanExp:
    def __init__(self, post_mean, post_mean2, post_sd):
        self.post_mean = post_mean
        self.post_mean2 = post_mean2
        self.post_sd = post_sd

def posterior_mean_exp(betahat, sebetahat, log_pi, scale):
    assignment = np.exp(log_pi)
    assignment = assignment / assignment.sum(axis=1, keepdims=True)
    mu = 0
    post_assign = np.zeros((betahat.shape[0], scale.shape[0]))
    
    for i in range(betahat.shape[0]):
        post_assign[i,] = wpost_exp(x=betahat[i],
                                    s=sebetahat[i], 
                                    w=assignment[i,],
                                    scale=scale) 
    
    post_mean = np.zeros(betahat.shape[0])
    post_mean2 = np.zeros(betahat.shape[0])

    for i in range(post_mean.shape[0]):
        post_mean[i] = sum(post_assign[i, 1:] * my_etruncnorm(0,
                                                              np.inf,
                                                              betahat[i] - sebetahat[i]**2 * (1/scale[1:]), 
                                                              sebetahat[i]))
        post_mean2[i] = sum(post_assign[i, 1:] * my_e2truncnorm(0,
                                                                99999, #some weird warning for inf so just use something large enough for b
                                                                betahat[i] - sebetahat[i]**2 * (1/scale[1:]), 
                                                                sebetahat[i]))
        post_mean2[i] = max(post_mean[i], post_mean2[i])
    
    if np.any(np.isinf(sebetahat)):
        inf_indices = np.isinf(sebetahat)
        a = 1/scale[1:]
        # Equivalent of `post$mean[is.infinite(s)]` 
        post_mean[inf_indices] = np.sum(post_assign[inf_indices, 1:] / a, axis=1)

        # Equivalent of `post$mean2[is.infinite(s)]`
        post_mean2[inf_indices] = np.sum(2 * post_assign[inf_indices, 1:] / a**2, axis=1)

    # Calculate `post_sd`
    post_sd = np.sqrt(np.maximum(0, post_mean2 - post_mean**2))

    # Update `post_mean2` and `post_mean`
    post_mean2 = post_mean2 + mu**2 + 2 * mu * post_mean
    post_mean = post_mean + mu

    # Return the results as an instance of PosteriorMeanExpResult
    return PosteriorMeanExp(post_mean, post_mean2, post_sd)


In [40]:
# Example usage

expected_value =my_etruncnorm(0,2,3,1)

print(f"Expected value of the truncated normal distribution: {expected_value}")


Expected value of the truncated normal distribution: [1.48995049]


should equal to 1.48995049

In [4]:
my_e2truncnorm(0,2,3,1)

array([2.39340536])

should equal to  2.393405

In [52]:
betahat=  np.array([1,2,3,4,5])
sebetahat=np.array([1,0.4,5,1,1])
scale = autoselect_scales_mix_exp ( np.array([1,2,3,4,5]),  np.array([1,0.4,5,1,1]))

non_informativ = np.full( scale.shape[0], 1/ scale.shape[0])
n=betahat.shape[0]
log_pi =  np.log( np.tile(non_informativ, (n, 1)))

In [42]:
assignment = np.exp(log_pi)[0]


assignment = assignment /   sum(assignment)
print(assignment)
x=betahat[1]
s=sebetahat[1]

[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]


writing wpost_def

In [59]:
w=assignment
print(w)
wpost_exp ( x, s, w, scale)
temp_array =   np.zeros ( (betahat.shape[0], scale.shape[0]))
i=1
temp_array[i,] = wpost_exp ( x=betahat[i], s=sebetahat[i], w=w, scale=scale) 
print(temp_array[i,])

[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[3.53987758e-06 6.61493885e-06 1.06391083e-05 2.86976960e-05
 1.69165759e-04 1.40776600e-03 8.91255655e-03 3.45101678e-02
 8.34205613e-02 1.38160703e-01 1.72844260e-01 1.77093219e-01
 1.57939387e-01 1.28091688e-01 9.74010338e-02]


should  be array([3.53987758e-06, 6.61493885e-06, 1.06391083e-05, 2.86976960e-05,
       1.69165759e-04, 1.40776600e-03, 8.91255655e-03, 3.45101678e-02,
       8.34205613e-02, 1.38160703e-01, 1.72844260e-01, 1.77093219e-01,
       1.57939387e-01, 1.28091688e-01, 9.74010338e-02])

In [44]:
print(scale.shape[0] )
post_assign =   np.zeros ( (betahat.shape[0], scale.shape[0]))
for i in range(betahat.shape[0]):
    post_assign[i,] = wpost_exp ( x=betahat[i], s=sebetahat[i], w=np.exp(log_pi)[i,], scale=scale) 
print(post_assign)

post_assign[0,]== wpost_exp ( x=betahat[0], s=sebetahat[0], w=w, scale=scale) 


15
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[[6.52918851e-02 6.78002962e-02 6.90396965e-02 7.08629747e-02
  7.34774073e-02 7.70129806e-02 8.12078860e-02 8.48923837e-02
  8.58678052e-02 8.20342129e

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [45]:
post_mean = np.zeros(betahat.shape[0])
post_mean2 = np.zeros(betahat.shape[0])
print(post_mean)

for i in range(post_mean.shape[0]):
    post_mean[i]=  sum(    post_assign[i,1:] *  my_etruncnorm(0,
                                             np.inf,
                                             betahat[i]- sebetahat[i]** 2 *(1/scale[1:]) , 
                                             sebetahat[i] 
                                             )
                       )
    post_mean2[i] =  sum ( post_assign[i,1:] *my_e2truncnorm(0,
                                             99999, #some weird warning for inf so just use something large enought for b
                                             betahat[i]- sebetahat[i]** 2 *(1/scale[1:]) , 
                                             sebetahat[i] 
                                             )
                       )
     
    post_mean2[i]= max(post_mean[i],post_mean2[i])
print(post_mean)

print(betahat )

[0. 0. 0. 0. 0.]
[0.4836384  1.89328341 1.05670973 3.57009763 4.66379651]
[1 2 3 4 5]


In [60]:
n=betahat.shape[0]
p= scale.shape[0]
 
log_pi =  np.log( np.full( (n, p), 1/scale.shape[0]))
 
res = posterior_mean_exp(betahat, sebetahat, log_pi, scale)
print(res.post_mean)
print(res.post_mean2)

[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.4836384  1.89328341 1.05670973 3.57009763 4.66379651]
[ 0.59674218  3.75372025  4.34337321 13.89299102 22.81107216]


res.post_mean should be equal to [0.4836384 1.8932834 1.0567097 3.5700976 4.6637965]
res.post_mean2 should be equal to [ 0.5967422  3.7537202  4.3433782 13.8929910 22.8110722]

Rscript to double check code aboce




x  <- c( 1,2,3,4,5 )
s  <- c( 1,0.4,5,1,1 )
 
mu <- 0
scale <-  autoselect_scales_mix_exp ( c(1,2,3,4,5 ),  c(  1,0.4,5,1,1 ))

a=1/scale[-1]
post <- list()


wpost_exp <- function(x, s, w,g) {
  
  # assuming a[1 ]=0
  if (w[1] == 1) {
    
    return(c(1, rep(0,length(g$scale) - 1))  )
  }
  
  
  lf <- dnorm(x, 0, s, log = TRUE)
  lg <- log(a) + s^2 * a^2 / 2 - a * x + pnorm(x / s - s * a, log.p = TRUE)
  
  
  
  log_prob = c(lf, lg)
  bmax = max(log_prob)
  log_prob = log_prob - bmax
  wpost <- w* exp( log_prob) / (sum(w *exp(log_prob)))
  
  #wpost <- w*c(exp(lf), exp( lg)) / (sum(w *c(exp(lf), exp( lg))))#underflow here
  
  
  
  
  
  return(wpost)
}

assignment <- matrix(1/length(scale), nrow = length(x), ncol = length(scale))

post_assignment <- do.call(rbind,
               lapply(1:length(x),
                      function(i)
                        wpost_exp(x = x[i],
                                  s = s[i],
                                  w = assignment [i, ],
                                  g =  fit$g)
               )
)

post$mean  <- apply( post_assignment[,-1] *ashr:: my_etruncnorm(0,
                                                                Inf,
                                                                x - s^2 %*% t(a),
                                                                s),
                     1,
                     sum)

post$mean2 <- apply( post_assignment[,-1] *ashr:: my_e2truncnorm(0,
                                                                 Inf,
                                                                 x - s^2 %*% t(a),
                                                                 s),
                     1,
                     sum)
post$mean2 <- pmax(post$mean2, post$mean^2)



if (any(is.infinite(s))) {
  post$mean[is.infinite(s)]  <-  apply(
    post_assignment[is.infinite(s), -1] / a,
    1,
    sum)
  post$mean2[is.infinite(s)] <-  apply(
    2*post_assignment[is.infinite(s), -1] / a^2,
    1,
    sum)
}
post$sd <- sqrt(pmax(0, post$mean2 - post$mean^2))

post$mean2 <- post$mean2 + mu^2 + 2 * mu * post$mean
post$mean  <- post$mean + mu

